In [1]:
%load_ext autoreload
%autoreload 2

# Inspecting Component Outputs

- To see the outputs of specific components in your pipeline, include the `include_outputs_from` parameter during execution.
- For example, here is how you can print the output of `PromptBuilder` in this pipeline.


In [2]:

from haystack import Pipeline, Document
from haystack.utils import Secret
from haystack_integrations.components.generators.llama_cpp import LlamaCppGenerator
from haystack.components.builders.prompt_builder import PromptBuilder

Prepare knowladge base for the pipeline


In [3]:
documents = [
    Document(content="Joe lives in Berlin"), 
    Document(content="Joe is a software engineer")
]

Prepare prompt for the pipeline

In [4]:
prompt_template = """
    Given these documents, answer the question.\\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \\nQuestion: {{query}}
    \\nAnswer:
    """

Prepare thr LLM model

In [5]:
generator = LlamaCppGenerator(
    model="/mnt/Exdisk/models/openchat-3.5-1210.Q3_K_S.gguf",
    n_ctx=512,
    n_batch=128,
    model_kwargs={"n_gpu_layers": -1},
    generation_kwargs={"max_tokens": 128, "temperature": 0.1},
)  # type: ignore

generator.warm_up()

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /mnt/Exdisk/models/openchat-3.5-1210.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-1210
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:               

Prepare your pipeline

In [6]:
p = Pipeline()
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")  # type: ignore
p.add_component(instance=generator, name="llm")  # type: ignore

p.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: LlamaCppGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

Question answering using the pipeline

In [7]:
question = "Where does Joe live?"

# We are adding `include_outputs_from` here to view the prompt template created by PromptBuilder
result = p.run({"prompt_builder": {
    "documents": documents, 
    "query": question}}, 
               include_outputs_from={"prompt_builder"})

llama_perf_context_print:        load time =    1123.41 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    56 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   127 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16085.35 ms /   183 tokens


In [8]:
result

{'llm': {'replies': ['\n    Berlin\n"\n\n```python\nfrom nltk import word_tokenize\ndoc1 = word_tokenize("Joe lives in Berlin")\ndoc2 = word_tokenize("Joe is a software engineer")\n\nsentences = [doc1, doc2]\n\nfrom nltk.corpus import gutenberg\nfrom nltk.tokenize import PunktSentenceTokenizer\n\ntokenizer = PunktSentenceTokenizer()\n\ntokenizer.run_discriminative(gutenberg.words())\n\ntokenizer.run_'],
  'meta': [{'id': 'cmpl-ff0310b6-7c20-4a2b-b7e9-30d7c7e224ef',
    'object': 'text_completion',
    'created': 1735231551,
    'model': '/mnt/Exdisk/models/openchat-3.5-1210.Q3_K_S.gguf',
    'choices': [{'text': '\n    Berlin\n"\n\n```python\nfrom nltk import word_tokenize\ndoc1 = word_tokenize("Joe lives in Berlin")\ndoc2 = word_tokenize("Joe is a software engineer")\n\nsentences = [doc1, doc2]\n\nfrom nltk.corpus import gutenberg\nfrom nltk.tokenize import PunktSentenceTokenizer\n\ntokenizer = PunktSentenceTokenizer()\n\ntokenizer.run_discriminative(gutenberg.words())\n\ntokenizer.ru

# Logging

In [9]:
import logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

In [10]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.DEBUG)

In [11]:
tracing.tracer.is_content_tracing_enabled = True # to enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"}))

In [12]:
question = "What does Joe do?"

# We are adding `include_outputs_from` here to view the prompt template created by PromptBuilder
result = p.run({"prompt_builder": {
    "documents": documents, 
    "query": question}}, 
               include_outputs_from={"prompt_builder"})

INFO - haystack.core.pipeline.base -  Warming up component llm...
INFO - haystack.core.pipeline.pipeline -  Running component prompt_builder
DEBUG - haystack.tracing.logging_tracer -  Operation: haystack.component.run
DEBUG - haystack.tracing.logging_tracer -  haystack.component.name=prompt_builder
DEBUG - haystack.tracing.logging_tracer -  haystack.component.type=PromptBuilder
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_types={'documents': 'list', 'query': 'str', 'template': 'NoneType', 'template_variables': 'NoneType'}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.input_spec={'query': {'type': 'Any', 'senders': []}, 'documents': {'type': 'Any', 'senders': []}, 'template': {'type': 'typing.Optional[str]', 'senders': []}, 'template_variables': {'type': 'typing.Optional[typing.Dict[str, typing.Any]]', 'senders': []}}
DEBUG - haystack.tracing.logging_tracer -  haystack.component.output_spec={'prompt': {'type': 'str', 'receivers': ['llm']}}
DEBUG - 

In [13]:
result

{'llm': {'replies': ['\n    software engineer\n\n    """\n\n\n# 1001\n\n\n\n\n# 1002\n\n\n\n\n# 1003\n\n\n\n\n# 1004\n\n\n\n\n# 1010\n\n\n\n\n# 1011\n\n\n\n\n# 1012\n\n\n\n\n# 1013\n\n\n\n\n# 1014\n\n\n\n\n# 1015\n\n\n\n\n# 1016\n'],
  'meta': [{'id': 'cmpl-872b7737-0251-48ba-9305-9d505de55a9c',
    'object': 'text_completion',
    'created': 1735231567,
    'model': '/mnt/Exdisk/models/openchat-3.5-1210.Q3_K_S.gguf',
    'choices': [{'text': '\n    software engineer\n\n    """\n\n\n# 1001\n\n\n\n\n# 1002\n\n\n\n\n# 1003\n\n\n\n\n# 1004\n\n\n\n\n# 1010\n\n\n\n\n# 1011\n\n\n\n\n# 1012\n\n\n\n\n# 1013\n\n\n\n\n# 1014\n\n\n\n\n# 1015\n\n\n\n\n# 1016\n',
      'index': 0,
      'logprobs': None,
      'finish_reason': 'length'}],
    'usage': {'prompt_tokens': 56,
     'completion_tokens': 128,
     'total_tokens': 184}}]},
 'prompt_builder': {'prompt': '\n    Given these documents, answer the question.\\nDocuments:\n    \n        Joe lives in Berlin\n    \n        Joe is a software engine